# imports ...

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')


In [2]:
import time
import os

from datetime import date
from datetime import timedelta
from datetime import datetime

import pickle

import json

import pandas as pd
import numpy as np
import random 

from handle_data.read_data import load_candles

from backtesting import Strategy
from backtesting import Backtest

import pandas_ta as ta
import jinja2

Loading BokehJS ...

In [3]:
from backtesting_sampler.sampler import process_heatmap

# Configuration

In [4]:
symbol = "BTC/USDT"
asset_factor = 1_000_000
timeframe='1d'
exchange='Binance'

experiment_nickname = "Range"

In [5]:
history_days = 10 * 30
start_date = (date.today() - timedelta(days=history_days)).strftime('%Y-%m-%d')
end_date=None

if experiment_nickname == "Bullrun":
    start_date = "2019-11-01"
    #end_date="2021-07-22"
    #end_date="2021-04-14"
elif experiment_nickname == "Bear":
    start_date = "2021-11-12"
    end_date="2022-11-12"
elif experiment_nickname == "Range":
    start_date = "2022-06-19"
    end_date="2023-03-12"
    
if not end_date:
    end_date=date.today().strftime('%Y-%m-%d')
    
print("start_date:",start_date)
print("end_date:",end_date)

start_date: 2022-06-19
end_date: 2023-03-12


In [6]:
timestamp = datetime.now().strftime('%Y-%m-%d_%H%M')
base_experiment_name=(exchange + '_' + symbol + '_' + timeframe + '_' + start_date + '_' + start_date + '_' + timestamp).replace('/', '-')

In [7]:
data = load_candles(exchange, symbol, timeframe=timeframe, factor=asset_factor, start_date=start_date, end_date=end_date)

reading /home/jovyan/work/data/BTC-USDT/1d/BTC-USDT_1d_Binance.pickle


# Class

In [8]:
class DCA_Conditional_Buy_LR_with_TrailingStop(Strategy):  
    # DCA + Moving Stop Loss
    
    band_lenght = 13
    band_mult = 1
    
    intial_sl_factor = 0.0
    minimal_benefit_to_start_trailing = 1.003
    
    sleep_by = 0

    reinvest = 2 # 0 = use only initial, 1 = reinvest all, 2 = use dca_budget reinvesting all
    stop_loose_changes = 0
    i_cash = 1_000
    
    buy_all_days = 0
    dca_budget = 100
    lr_buy_longitude = 3
    
    
    def lb_close(self,data): #lower_band_stdev_formula
        dema = ta.dema(close = data.Close.s, length=self.band_lenght).to_numpy()
        stdev = ta.stdev(close = data.Close.s, length=self.band_lenght).to_numpy()
        lower_band = dema - ( self.band_mult * stdev )
        return lower_band
    
    def lb_low(self,data): # lower_band_stdev_formula_on_low
        dema = ta.dema(close = data.Low.s, length=self.band_lenght).to_numpy()
        stdev = ta.stdev(close = data.Low.s, length=self.band_lenght).to_numpy()
        lower_band = dema - ( self.band_mult * stdev )
        return lower_band

    
    def LR(self,data):
        # https://greyhoundanalytics.com/blog/custom-indicators-in-backtestingpy/
        linreg = ta.linreg(close = data.Close.s, length=self.lr_buy_longitude, angle=True )
        return linreg.to_numpy()
    

    
    def calculate_size(self):
        if self.reinvest==0:
            if self.equity < self.i_cash:
                cash = self.equity
            else:
                cash = self.i_cash
            size = int((cash / self.data.Close[-1]) * 0.99 )
            #print(f"buy with limit {size} = {self.i_cash} / {self.data.Close[-1]}")
        elif self.reinvest==1:
            size = int( (self.equity / self.data.Close[-1]) * 0.99 )
            #print(f"buy no limit {size} = {self.equity} / {self.data.Close[-1]} ")
        elif self.reinvest==2:
            if self.equity < self.dca_budget:
                cash = self.equity
            else:
                cash = self.dca_budget
            size = int( ( cash / self.data.Close[-1]) * 0.99 )

        return size
    

    
    def init(self):
        self.day_of_week = self.I(lambda x: x,self.data.Close.s.index.dayofweek,plot = False,)
        self.lr_buy = self.I(self.LR, self.data,plot = False)
        self.lower_band = self.I(self.lb_close, self.data)
        self.lb_low = self.I(self.lb_low, self.data)

        self.current_stop_loss = 0 
        self.i_cash = self.equity
        self.dca_bought = False
        
    def do_buy(self,sl):
        self.current_stop_loss = sl * self.intial_sl_factor
        size=self.calculate_size()
        if size>0:
            self.buy(size=size,sl=self.current_stop_loss) 
        
    def next(self):

        new_stop = self.lower_band[-1]
        new_stop_price = self.lb_low[-1]
        
        if self.reinvest in [0,1]:
            if not self.position:
                if not self.should_sleep(): 
                    self.do_buy(new_stop)
                    self.sleep_by = random.random() * 7 * 24 * 60 
        else:
            if self.day_of_week[-1] == 1 or self.buy_all_days: 
                if self.dca_bought == False and self.lr_buy[-1]>0:
                    self.do_buy(new_stop)
                    self.dca_bought == True
            else:
                self.dca_bought == False
        
         
        if new_stop > self.current_stop_loss:
            self.current_stop_loss = new_stop
            for trade in self.trades:
                if new_stop_price > trade.entry_price * self.minimal_benefit_to_start_trailing:
                    old_stop = trade.sl
                    if self.data.Close[-1] > new_stop:
                        trade.sl = new_stop
                        #print(f"I'm recreating the stop loss {old_stop}, new: {new_stop}. Entry price {self.trades[0].entry_price}")
            else:
                #print(f"ERROR, I'm trying to create the stop loss {old_stop}, new: {new_stop}. Entry price {self.trades[0].entry_price}")
                pass

# Optimization

In [9]:
start_time = time.time()

In [10]:
fb = [1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987]

In [11]:
bt = Backtest(data, DCA_Conditional_Buy_LR_with_TrailingStop, cash=1600, commission=.001)
optimize = True
heatmap = None

if optimize:
     stats, heatmap = bt.optimize(return_heatmap= True,maximize="Return [%]",
                        band_lenght=[3,9,10,11,12,13,21,34],
                        band_mult=[1,2],
                        #sl_activation_margin=[1.003, 1.004, 1.005, 1.006, 1.007, 1.008, 1.009, 1.01, 1.012 ],
                        minimal_benefit_to_start_trailing=[1.003],
                        reinvest = [2],
                        dca_budget = [100,200,500,1000,1600],
                        buy_all_days=[0,1],
                        lr_buy_longitude=[*range(2, 7, 1)],

    )
else:
    # fast run
    stats = bt.run(band_lenght=34,band_mult=2,sl_activation_margin=1.003,reinvest=2,dca_budget=100,buy_all_days=1,lr_buy_longitude=5)



Backtest.optimize:   0%|          | 0/10 [00:00<?, ?it/s]

In [12]:
end_time = time.time()
duration = (end_time - start_time)
print("--- %s seconds ---" % duration )

--- 2.761631727218628 seconds ---


# Results

In [13]:
stats._strategy

<Strategy DCA_Conditional_Buy_LR_with_TrailingStop(band_lenght=9,band_mult=1,minimal_benefit_to_start_trailing=1.003,reinvest=2,dca_budget=1600,buy_all_days=0,lr_buy_longitude=6)>

In [14]:
stats

Start                     2022-06-19 00:00:00
End                       2023-03-12 00:00:00
Duration                    266 days 00:00:00
Exposure Time [%]                   65.917603
Equity Final [$]                  2213.301929
Equity Peak [$]                   2513.757579
Return [%]                          38.331371
Buy & Hold Return [%]                6.917031
Return (Ann.) [%]                   55.827598
Volatility (Ann.) [%]               48.186159
Sharpe Ratio                         1.158582
Sortino Ratio                        3.331087
Calmar Ratio                         4.367113
Max. Drawdown [%]                  -12.783638
Avg. Drawdown [%]                   -4.005411
Max. Drawdown Duration      108 days 00:00:00
Avg. Drawdown Duration       16 days 00:00:00
# Trades                                    5
Win Rate [%]                             80.0
Best Trade [%]                      36.577839
Worst Trade [%]                    -11.631655
Avg. Trade [%]                    

In [15]:
bt.plot()

GridPlot(id='p1309', ...)

In [16]:
#from backtesting.lib import plot_heatmaps
#plot_heatmaps(heatmap)

In [17]:
if not(heatmap is None):
    heatmap_df = process_heatmap(heatmap)
else:
    heatmap_df = pd.DataFrame()

heatmap_df

,band_lenght,band_mult,minimal_benefit_to_start_trailing,reinvest,dca_budget,buy_all_days,lr_buy_longitude,Return [%]
144,9,1,1.003,2,1600,0,6,38.331371
244,10,1,1.003,2,1600,0,6,38.065148
344,11,1,1.003,2,1600,0,6,35.737490
444,12,1,1.003,2,1600,0,6,33.583091
134,9,1,1.003,2,1000,0,6,28.239401
...,...,...,...,...,...,...,...,...
729,34,1,1.003,2,500,1,6,-12.985585
769,34,2,1.003,2,200,1,6,-13.208277
719,34,1,1.003,2,200,1,6,-13.208277
749,34,1,1.003,2,1600,1,6,-14.172719


# Write results

In [18]:
base_results_dir = 'results'

In [19]:
srt_strategy_configuration = str(stats._strategy)
str_strategy_name = srt_strategy_configuration[:srt_strategy_configuration.find("(")]
parameters = srt_strategy_configuration[srt_strategy_configuration.find("(") + 1:-1]

In [20]:
directory = base_results_dir + '/' + str_strategy_name + '/' + experiment_nickname + '/' + base_experiment_name
directory

'results/DCA_Conditional_Buy_LR_with_TrailingStop/Range/Binance_BTC-USDT_1d_2022-06-19_2022-06-19_2024-01-20_1404'

In [21]:
if not os.path.exists(directory):
    os.makedirs(directory)

In [22]:
bt.plot(filename=directory+'/bt_interactive_plot.html')

GridPlot(id='p1649', ...)

In [23]:
file_name = directory + "/heatmap_df"   

with pd.ExcelWriter(file_name + ".xlsx", engine='xlsxwriter') as writer:
    heatmap_df.to_excel(writer, index=False)
heatmap_df.to_markdown(file_name + ".md")

file_name = directory + "/trades"   

with pd.ExcelWriter(file_name + ".xlsx", engine='xlsxwriter') as writer:
    stats._trades.to_excel(writer, index=False)
stats._trades.to_markdown(file_name + ".md")

file_name = directory + "/equity_curve"   

with pd.ExcelWriter(file_name + ".xlsx", engine='xlsxwriter') as writer:
    stats._equity_curve.to_excel(writer, index=False)
stats._equity_curve.to_markdown(file_name + ".md")   

In [24]:
str_stats = str(stats)

In [25]:
heatmap_md = heatmap_df[0:10].to_markdown()

In [26]:
strat_return = stats['Return [%]']
buy_and_hold = stats['Buy & Hold Return [%]']
equity_init = stats._equity_curve[0:1]['Equity'].iloc[0]
equity_final = stats['Equity Final [$]']

In [27]:
jinja2_templates_dir = 'templates'
jinja2_template_file = 'single_result.md'

# write results
filename = f"{directory}/index.md"
environment = jinja2.Environment(loader=jinja2.FileSystemLoader(jinja2_templates_dir))
template = environment.get_template(jinja2_template_file)
rendered = template.render(str_strategy_name=str_strategy_name, experiment_nickname= experiment_nickname,symbol=symbol, exchange=exchange, start_date=start_date, end_date=end_date, timeframe=timeframe,
                           str_stats=str_stats, 
                           heatmap_md=heatmap_md, parameters=parameters.split(','), 
                           strat_return=strat_return, buy_and_hold=buy_and_hold, 
                           equity_init=equity_init,
                           equity_final=equity_final)

with open(filename, mode="w", encoding="utf-8") as message:
    message.write(rendered)
    print(f"Wrote: {filename}")

Wrote: results/DCA_Conditional_Buy_LR_with_TrailingStop/Range/Binance_BTC-USDT_1d_2022-06-19_2022-06-19_2024-01-20_1404/index.md
